In [1]:
import numpy as np
import random
import json
import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Flatten, Embedding

import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
#nltk.download('punkt', download_dir='/home/chamath/Documents/PyProjects/ChatBot')
#nltk.download('wordnet', download_dir='/home/chamath/Documents/PyProjects/ChatBot')

nltk.data.path.append('/home/chamath/Documents/PyProjects/ChatBot')

In [3]:
words = []
classes = []
documnets = []
ignore_word = ['?', '!']

In [4]:
#data_file = open('intents.json').read()
#intents = json.load(data_file)
with open('intents.json', 'r') as data_file:
    intents = json.load(data_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documnets.append((w, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_word]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documnets), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [7]:
training = []
output_empty = [0] * len(classes)

for doc in documnets:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
 
random.shuffle(training)

training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training data created")

Training data created


<ipython-input-7-509f833c5f7b>:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [18]:
print(len(train_x[0]))
print(len(train_x[46]))
print(type(train_x))

88
88
<class 'list'>


In [19]:
train_x = np.array(train_x)
train_y = np.array(train_y)
train_x.shape

(47, 88)

In [26]:
len(train_x[0])
print(train_y.shape)

(47, 9)


In [33]:
model = Sequential()

model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
#model.add(Embedding(89, 10, input_length=len(train_x[0])))
#model.add(LSTM(8, return_sequences=True, input_shape=(88, 10)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [34]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
model.fit(np.array(train_x), np.array(train_y), epochs=50, batch_size=5, verbose=1)

Epoch 1/50
10/10 [==============================] - 3s 10ms/step - loss: 2.2139 - accuracy: 0.0927
Epoch 2/50
10/10 [==============================] - 0s 12ms/step - loss: 2.1567 - accuracy: 0.1792
Epoch 3/50
10/10 [==============================] - 0s 8ms/step - loss: 2.0523 - accuracy: 0.3449
Epoch 4/50
10/10 [==============================] - 0s 8ms/step - loss: 2.0389 - accuracy: 0.3580
Epoch 5/50
10/10 [==============================] - 0s 9ms/step - loss: 1.9858 - accuracy: 0.2616
Epoch 6/50
10/10 [==============================] - 0s 8ms/step - loss: 1.9274 - accuracy: 0.4934
Epoch 7/50
10/10 [==============================] - 0s 10ms/step - loss: 1.8578 - accuracy: 0.3662
Epoch 8/50
10/10 [==============================] - 0s 8ms/step - loss: 1.8321 - accuracy: 0.4875
Epoch 9/50
10/10 [==============================] - 0s 9ms/step - loss: 1.7299 - accuracy: 0.4907
Epoch 10/50
10/10 [==============================] - 0s 10ms/step - loss: 1.5961 - accuracy: 0.5411
Epoch 11/50
10/

In [36]:
model.save('chatbot_model_model2.h5')